In [ ]:
# # Pull Data

# # Snowflake , Get Data

# import snowflake.connector
# from snowflake.sqlalchemy import URL
# from sqlalchemy import create_engine
# # Sql Engine, where you input connection paramaters, User Name, password etc. Stuff you see in the snowflake gui top right. 

# # snowflake function
# def get_data_from_snowflake(sql):

#     engine = create_engine(URL(
#         account = 'cdm',
#         user = 'kchun',
#         password = 'Monarch219!',
#         database = 'EDWDBPRD',
#         #schema = 'public',
#         warehouse = 'ANALYTICS_PROD_WH',
#         role='DATA_SCIENCE_PROD_ADMIN_ROLE',
#         authenticator='externalbrowser'
#     ))

#     print('[INFO] connecting to Snowflake')
#     connection = engine.connect()
#     print('[INFO] connected, executing query...')

#     try:
#         df = pd.read_sql_query(sql, engine)
#         df.columns = df.columns.str.upper()
#         df.rename(index=str, columns={'INQUIRY': 'Inquiry','PLACEMENT':'placement'}, inplace=True)
#         print('[INFO] query executed')
#     finally:
#         connection.close()
#         engine.dispose()

#     return(df)

In [1]:
import pandas as pd
import numpy as np

In [3]:
# record prob output drawn to decide the argmax into a dataframe
prob_df = pd.DataFrame(columns=['p0','p1','p2','p3', \
          'p4']);print(prob_df);print(0)

Empty DataFrame
Columns: [p0, p1, p2, p3, p4]
Index: []
0


In [9]:
prob_df

,p0,p1,p2,p3,p4


In [ ]:
 #this is all duplicated below
print("\n Bayesian Bandit using Thompson Sampling \n")
print("Goal is to maximize the attempts on the highest converting variant")


N = 5  #Number of Variants


means = np.array([0.7, 0.3, 0.4, 0.5, .55])  # Define the conversion rates, unknown in real life

probs = np.zeros(N)  # random sample from posterior probability distribution for each machine (EV would be mean?)
            # this simply determines which one to pull(the handle of machine with the higest one of these will be pulled)
S = np.zeros(N, dtype=np.int)  # number successes each machine
F = np.zeros(N, dtype=np.int)  # number failures each machine
rnd = np.random.RandomState(221)  # for machine payouts and Beta

Bayesian Bandit using Thompson Sampling 

Goal is to maximize the attempts on the highest converting variant

In [ ]:
probs

Out[7]: array([0., 0., 0., 0., 0.])

In [ ]:
S

Out[8]: array([0, 0, 0, 0, 0])

In [ ]:
F

Out[9]: array([0, 0, 0, 0, 0])

In [ ]:
rnd.beta(1,1) #will always be 0.14278824800173925 and then 0.873104102698192...

Out[11]: 0.873104102698192

In [ ]:
np.mean(rnd.beta(1,1,30000000)) #usually will have 3 zeros to right of 5 (due to pulling 30 MILLION numbers from beta distrib.)

Out[34]: 0.5000021114952294

In [ ]:
np.mean(rnd.beta(1,1,3000)) #usually has 1 zero to right of 5 (due to only pulling 3k numbers from beta distrib.)

Out[32]: 0.49919486502068305

***
Note: no matter how many trials and samples (30000000 above) we use, the mean estimates of the lower converting handles are never very accurate -- that is b/c they are pulled so little - which is a good thing!

Next step is to set a random number of days that the win comes in at (loss always comes in after 100 days)

I can loop through a dict of all previous pulls (containing how long that win or loss will take to come in) and decrement it for each 1000 trials (assuming we send 1000 inquiries a day) -- when it hits zero, I pull it into the posterior beta(S,F) calc

show how even if distribution is not same bandit still works while A/B can NOT

splash A/B funnel C/D 4 combos
***

In [ ]:
# record prob output drawn to decide the argmax into a dataframe

print("\n Bayesian Bandit using Thompson Sampling \n")
print("Goal is to maximize the attempts on the highest converting variant")


N = 5  #Number of Variants

trials = 20   #how many total pulls do we want to do?

trial_data =[]  # array of trial #'s (for display at end)

machine_data=[] # array of which handles we pulled on each trial (for display at end)

win_loss=[] #array of whether we won or lost on each trial (for display at end)



means = np.array([0.70, 0.65, 0.60, 0.55, 0.50])  # Define the actual conversion rates for each machine (unknown in real life)



probs = np.zeros(N) #temp storage for single trial: random sample from posterior probability distribution for each machine(EV = the mean?)
   #NOTE: this simply determines which machine to pull(the handle of machine with the higest one of these will be pulled)

prob_df = pd.DataFrame(columns=['p0','p1','p2','p3','p4']) #holds the history of probs (above) from each trial



beta_mean = np.zeros(N) #temporary storage for a single trial: I will take the mean of a random sample from the same posterior distributions used above for probs (just to show it)

beta_mean_df = pd.DataFrame(columns=['p0','p1','p2','p3','p4']) #holds the history of beta_mean's (above) from each trial


S = np.zeros(N, dtype=np.int)  # running count of successes for each machine
F = np.zeros(N, dtype=np.int)  # running count of failures for each machine
pulls = np.zeros(N, dtype=np.int) # running count of pulls for each machine (just used at end for display)

rnd = np.random.RandomState()  #random series for machine payouts and Beta (put in a number to replicate series)




show_each_trial = 1 #change to 1 if you want to print each trial (otherwise only prints last one)

def prints(*args, **kwargs):
    if show_each_trial == 1:
        if kwargs=="":print(*args, **kwargs, end="")
        else: print(*args, **kwargs)
    elif trial >= trials-1:
        if kwargs=="":print(*args, *kwargs, end="")
        else: print(*args, **kwargs)

def samples():
  if show_each_trial == 1:
    return 30000000
  elif trial >= trials-1:
    return 30000000
  else:
    return 3 #no need for precision if we aren't going to display the mean for this trial
 
for trial in range(trials):
    
    prints("\nTrial " + str(trial))
    
    trial_data.append(trial) #append this trial # to the trial_data array we defined above
    
    #for the new trial, clear out the probabilities we use to decide which machine to pull (see above desc for same variable)
    probs = np.zeros(N)
    
    #pull from the posterior probability distribution for each machine to get the prob for it (again, just used to pick handle)
    for i in range(N): 
      
      probs[i] = rnd.beta(S[i] + 1, F[i] + 1) #since we are using a beta(1,1) prior, this is the posteror prob.
    
      beta_mean[i] = np.mean(rnd.beta(S[i] + 1, F[i] + 1, samples())) #last number is how many samples to pull to calc mean

      global prob_df  
      if i == 4: #last machine
        columns = ['p0','p1','p2','p3','p4']  
        
        df = pd.DataFrame(probs, index=columns).T 
        prob_df = prob_df.append(df, ignore_index = True) #save the probs for this trial
        
        df = pd.DataFrame(beta_mean, index=columns).T
        beta_mean_df = beta_mean_df.append(df, ignore_index = True) #save the means of the dist's used to get the probs
        
      else:
        pass
  
    prints("means of posterior probability dist (this should get closer to actual over time)")
    for i in range(N):
        prints('%0.4f ' % beta_mean[i], end='')
        
    prints("")
    
    prints("sampling probs =  ")
    for i in range(N):
      prints("%0.4f  " % probs[i], end="")

      
    prints("")

    machine = np.argmax(probs)
    prints("Playing machine " + str(machine))
    
    machine_data.append(machine)
    # Updates success and fails in real life we can can update conversions , clicks, or whatever success of chosen variant
    
    p = rnd.random_sample()  # [0.0, 1.0)
    prints(p)
    if p < means[machine]:
          prints(" -- win")
          S[machine] += 1
          win_loss.append('W')
    else:
          prints(" -- lose")
          F[machine] += 1
          win_loss.append('L')

    prints("\nSuccess vector: ", end="")
    prints(S)
    prints("Failure vector: ", end="")
    prints(F)
    prints("Total pulls: ", end="")
    pulls=[S[i]+F[i] for i in range(len(S))]
    prints(pulls)
    


In [ ]:
#put all the data together

trial_df=pd.DataFrame(trial_data,columns=['trials']).reset_index(drop=True)

machine_df=pd.DataFrame(machine_data,columns=['machine']).reset_index(drop=True)

win_loss_df=pd.DataFrame(win_loss,columns=['win/loss']).reset_index(drop=True)

final = pd.concat([trial_df, machine_df, win_loss_df, prob_df], axis=1, sort=False)

In [ ]:
final

In [ ]:
def main():
    print(0)

if __name__ == "__main__":
  main()
    

In [ ]:
final